<a href="https://colab.research.google.com/github/WeydisonAndrade/ans-data-pipeline/blob/main/Transforma%C3%A7%C3%A3o_e_valida%C3%A7%C3%A3o_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#VALIDAÇÃO DE DADOS COM ESTRATÉGIAS DIFERENTES

In [1]:
import pandas as pd


In [3]:
from google.colab import files

uploaded = files.upload()

Saving consolidado_despesas.zip to consolidado_despesas.zip


In [6]:
#VALIDAÇÃOD E CNPJ

import re

def validar_cnpj(cnpj: str) -> bool:
    """
    Valida CNPJ verificando:
    - Formato
    - Quantidade de dígitos
    - Dígitos verificadores
    """

    # Remove tudo que não for número
    cnpj = re.sub(r'\D', '', cnpj)

    # Verifica se tem 14 dígitos
    if len(cnpj) != 14:
        return False

    # Elimina CNPJs com todos os dígitos iguais
    if cnpj == cnpj[0] * 14:
        return False

    # Pesos para cálculo dos dígitos
    pesos_1 = [5, 4, 3, 2, 9, 8, 7, 6, 5, 4, 3, 2]
    pesos_2 = [6] + pesos_1

    def calcular_digito(cnpj_parcial, pesos):
        soma = sum(int(d) * p for d, p in zip(cnpj_parcial, pesos))
        resto = soma % 11
        return '0' if resto < 2 else str(11 - resto)

    # Cálculo dos dígitos verificadores
    digito_1 = calcular_digito(cnpj[:12], pesos_1)
    digito_2 = calcular_digito(cnpj[:12] + digito_1, pesos_2)

    # Verifica se os dígitos calculados conferem
    return cnpj[-2:] == digito_1 + digito_2


In [9]:
import pandas as pd

df = pd.read_csv(list(uploaded.keys())[0])
df.head()


,Trimestre,Ano,ValorDespesas
0,2025Q3,2025,8.223670e+05
1,2025Q3,2025,2.927156e+08
2,2025Q3,2025,2.300688e+08
3,2025Q3,2025,2.315562e+08
4,2025Q3,2025,2.289016e+08


In [10]:
import os
os.getcwd()
os.listdir()


['.config', 'consolidado_despesas.zip', 'sample_data']

In [11]:
import re

def validar_cnpj(cnpj: str) -> bool:
    cnpj = re.sub(r'\D', '', str(cnpj))

    if len(cnpj) != 14 or cnpj == cnpj[0] * 14:
        return False

    pesos_1 = [5,4,3,2,9,8,7,6,5,4,3,2]
    pesos_2 = [6] + pesos_1

    def calc_digito(cnpj_parcial, pesos):
        soma = sum(int(d) * p for d, p in zip(cnpj_parcial, pesos))
        resto = soma % 11
        return '0' if resto < 2 else str(11 - resto)

    d1 = calc_digito(cnpj[:12], pesos_1)
    d2 = calc_digito(cnpj[:12] + d1, pesos_2)

    return cnpj[-2:] == d1 + d2


In [19]:
df.columns = (
    df.columns
      .str.strip()       # remove espaços
      .str.lower()       # tudo minúsculo
      .str.replace(' ', '_')
)


In [20]:
df.columns
#OBS: NÃO HÁ uma coluna CNPJ nos arquivos consolidado_despezas.zip

Index(['trimestre', 'ano', 'valordespesas'], dtype='object')

In [22]:
#VALIDANDO VALORES NUMÉRICOS POSITIVOS

import math

def validar_numero_positivo(valor) -> bool:
    """
    Valida se o valor é numérico e estritamente positivo (> 0)
    """
    try:
        # Converte para float
        num = float(valor)

        # Rejeita NaN e infinito
        if math.isnan(num) or math.isinf(num):
            return False

        return num > 0
    except (ValueError, TypeError):
        return False


In [23]:
valores = [10, 0, -5, 3.7, '8', '0', '-2', 'abc', None, float('nan')]

for v in valores:
    print(v, validar_numero_positivo(v))


10 True
0 False
-5 False
3.7 True
8 True
0 False
-2 False
abc False
None False
nan False


In [24]:
df['valor_positivo'] = df['valor'].apply(validar_numero_positivo)


KeyError: 'valor'

In [25]:
df.columns


Index(['trimestre', 'ano', 'valordespesas'], dtype='object')

In [26]:
df['valor_positivo'] = df['valordespesas'].apply(validar_numero_positivo)


In [27]:
df.head()


,trimestre,ano,valordespesas,valor_positivo
0,2025Q3,2025,8.223670e+05,True
1,2025Q3,2025,2.927156e+08,True
2,2025Q3,2025,2.300688e+08,True
3,2025Q3,2025,2.315562e+08,True
4,2025Q3,2025,2.289016e+08,True


In [28]:
pd.options.display.float_format = '{:,.2f}'.format
df.head()


,trimestre,ano,valordespesas,valor_positivo
0,2025Q3,2025,"822,366.99",True
1,2025Q3,2025,"292,715,635.60",True
2,2025Q3,2025,"230,068,808.69",True
3,2025Q3,2025,"231,556,201.57",True
4,2025Q3,2025,"228,901,628.75",True


In [29]:
#VALIDAÇÃO DE RAZÃO SOCIAL NÃO VAZIA

import pandas as pd

def validar_razao_social(valor) -> bool:
    """
    Valida se a razão social é preenchida corretamente
    """
    if pd.isna(valor):
        return False

    if not isinstance(valor, str):
        return False

    # Remove espaços extras
    valor = valor.strip()

    if valor == '':
        return False

    # Opcional: tamanho mínimo
    return len(valor) >= 2


In [30]:
df.columns
#NÃO HÁ COMO APLICAR VALIDAÇÃO DE RAZÃO SOCIAL NÃO VAZIA PQ O ARQUIVO NÃO EXISTE NO DATAFRAME


Index(['trimestre', 'ano', 'valordespesas', 'valor_positivo'], dtype='object')

In [31]:
df.head()


,trimestre,ano,valordespesas,valor_positivo
0,2025Q3,2025,"822,366.99",True
1,2025Q3,2025,"292,715,635.60",True
2,2025Q3,2025,"230,068,808.69",True
3,2025Q3,2025,"231,556,201.57",True
4,2025Q3,2025,"228,901,628.75",True


In [32]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491407 entries, 0 to 491406
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   trimestre       491407 non-null  object 
 1   ano             491407 non-null  int64  
 2   valordespesas   491407 non-null  float64
 3   valor_positivo  491407 non-null  bool   
dtypes: bool(1), float64(1), int64(1), object(1)
memory usage: 11.7+ MB


In [34]:
if 'cnpj' in df.columns:
    df['cnpj_valido'] = df['cnpj'].apply(validar_cnpj)
else:
    print("Coluna 'cnpj' não encontrada. Validação de CNPJ não aplicada.")


Coluna 'cnpj' não encontrada. Validação de CNPJ não aplicada.


## Validação de CNPJ

### Contexto
Durante a análise do dataset, verificou-se que **não existe coluna de CNPJ**
nem coluna de Razão Social na base disponibilizada.

### Decisão Técnica
Optou-se por **não criar artificialmente** colunas de CNPJ ou Razão Social,
evitando a introdução de dados inexistentes ou inferidos.

A pipeline foi construída de forma defensiva, aplicando validações
apenas quando as colunas relevantes estiverem presentes no dataset.

### Justificativa
Criar ou inferir CNPJs poderia comprometer a integridade, rastreabilidade
e confiabilidade das análises, especialmente em contextos financeiros
ou regulatórios.

### Prós
- Mantém fidelidade absoluta aos dados de origem
- Evita dados artificiais ou inferidos
- Pipeline resiliente e reutilizável
- Alinhado a boas práticas de engenharia de dados

### Contras
- Não é possível realizar análises por entidade jurídica
- Limita validações cadastrais enquanto o dado não existir

### Observação
Caso futuras versões do dataset incluam CNPJ ou Razão Social,
as validações já estão prontas para serem aplicadas automaticamente.


In [35]:
#IMPORTANDO ARQUIVOS DE DADOS CADASTRAIS DE OPERADORAS ATIVAS

import os

os.listdir('/content')


['.config', 'consolidado_despesas.zip', 'sample_data']

In [36]:
import zipfile

zip_path = '/content/consolidado_despesas.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content')


In [37]:
import os

os.listdir('/content')


['.config',
 'consolidado_despesas.csv',
 'consolidado_despesas.zip',
 'sample_data']

In [40]:
import zipfile

zip_path = '/content/consolidado_despesas.zip'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.namelist()


In [41]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content')


In [42]:
import os
os.listdir('/content')


['.config',
 'consolidado_despesas.csv',
 'consolidado_despesas.zip',
 'sample_data']

In [44]:
import pandas as pd

df_cadop = pd.read_excel(
    '/content/Relatorio_cadop.xls',
    engine='xlrd'
)

df_cadop.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/Relatorio_cadop.xls'

In [45]:
import os

os.listdir('/content/var')


FileNotFoundError: [Errno 2] No such file or directory: '/content/var'

In [46]:
import os

for root, dirs, files in os.walk('/content'):
    if 'var' in dirs:
        print(os.path.join(root, 'var'))


In [47]:
for root, dirs, files in os.walk('/content'):
    for file in files:
        if 'cadop' in file.lower():
            print(os.path.join(root, file))


In [50]:
from google.colab import files

print("Please upload 'Relatorio_cadop.csv' or the zip file containing it.")
uploaded_cadop = files.upload()

Please upload 'Relatorio_cadop.csv' or the zip file containing it.


Saving Relatorio_cadop (1).csv to Relatorio_cadop (1) (1).csv


In [51]:
import os

os.listdir('/content')


['.config',
 'consolidado_despesas.csv',
 'consolidado_despesas.zip',
 'Relatorio_cadop (1).csv',
 'Relatorio_cadop (1) (1).csv',
 'sample_data']

In [52]:
import os

os.rename(
    '/content/Relatorio_cadop (1).csv',
    '/content/Relatorio_cadop.csv'
)


In [53]:
import os

os.listdir('/content')


['.config',
 'consolidado_despesas.csv',
 'consolidado_despesas.zip',
 'Relatorio_cadop.csv',
 'Relatorio_cadop (1) (1).csv',
 'sample_data']

In [54]:
df_cadop = pd.read_csv(
    '/content/Relatorio_cadop.csv',
    sep=';',
    encoding='latin1'
)


In [55]:
import pandas as pd

df_cadop = pd.read_csv(
    '/content/Relatorio_cadop.csv',
    sep=';',
    encoding='latin1'
)

df_cadop.head()


,REGISTRO_OPERADORA,CNPJ,Razao_Social,Nome_Fantasia,Modalidade,Logradouro,Numero,Complemento,Bairro,Cidade,UF,CEP,DDD,Telefone,Fax,Endereco_eletronico,Representante,Cargo_Representante,Regiao_de_Comercializacao,Data_Registro_ANS
0,419761,19541931000125,18 DE JULHO ADMINISTRADORA DE BENEFÃCIOS LTDA,NaN,Administradora de BenefÃ­cios,RUA CAPITÃO MEDEIROS DE REZENDE,274,NaN,PRAÃA DA BANDEIRA,AlÃ©m ParaÃ­ba,MG,36660000,32.00,"34,624,649.00",NaN,contabilidade@cbnassessoria.com.br,LUIZ HENRIQUE MARENDINO GONÃALVES,SÃCIO ADMINISTRADOR,6.00,2015-05-19
1,421545,22869997000153,2B ODONTOLOGIA OPERADORA DE PLANOS ODONTOLÃGI...,NaN,Odontologia de Grupo,RUA CATÃO,128,SALA 126,VILA ROMANA,SÃ£o Paulo,SP,5049000,11.00,"34,415,852.00",NaN,labmarisol@gmail.com,MARISOL BECHELLI,SÃCIO ADMINISTRADORA,4.00,2019-06-13
2,421421,27452545000195,2CARE OPERADORA DE SAÃDE LTDA.,NaN,Medicina de Grupo,RUA: BERNARDINO DE CAMPOS,230,1Âº ANDAR,CENTRO,Campinas,SP,13010151,19.00,"37,901,224.00",NaN,ans.plano@hospitalcare.com.br,RODRIGO PINHO RIBEIRO,REPRESENTANTE,5.00,2018-10-09
3,418030,13138885000131,A.P.S. ADMINISTRADORA DE BENEFÃCOS LTDA.,A.P.S. SAÃDE.,Administradora de BenefÃ­cios,RUA VOLUNTÃRIOS DA PÃTRIA,2525,CONJUNTO 143 - SALA 01,SANTANA,SÃ£o Paulo,SP,2401000,11.00,"45,223,468.00",NaN,diretoria@apssaude.com.br,PERCÃVEL GAETA,SÃ³CIO-ADMINISTRADOR E REPRESEN,4.00,2011-05-05
4,314668,17505793000101,ABERTTA SAÃDE - ASSOCIAÃÃO BENEFICENTE DOS ...,ABERTTA SAÃDE,AutogestÃ£o,AV. BERNARDO MONTEIRO,831,"Subsolo, 2Âº andar e 3Âº andar",SANTA EFIGÃNIA,Belo Horizonte,MG,30150281,31.00,"32,484,300.00","32,484,377.00",abertta.ans@arcelormittal.com.br,WERNER DUARTE DALLA,Diretor Presidente,4.00,1998-12-28


In [56]:
#Fazendo join entre o CSV consolidado e os dados cadastrais usando o CNPJ como chave

def normalizar_cnpj(cnpj):
    if pd.isna(cnpj):
        return None
    return (
        str(cnpj)
        .replace('.', '')
        .replace('/', '')
        .replace('-', '')
        .strip()
        .zfill(14)
    )


In [57]:
df_consolidado['cnpj_normalizado'] = df_consolidado['cnpj'].apply(normalizar_cnpj)
df_cadop['cnpj_normalizado'] = df_cadop['CNPJ'].apply(normalizar_cnpj)


NameError: name 'df_consolidado' is not defined

In [58]:
import pandas as pd

df_consolidado = pd.read_csv('consolidado_despesas.csv')


In [59]:
df_consolidado.head()
df_consolidado.columns


Index(['Trimestre', 'Ano', 'ValorDespesas'], dtype='object')

não é possível fazer um join usando CNPJ como chave, porque a chave não existe em um dos lados.

In [60]:
#AGRUPANDO DADOS POR RAZÃO SOCIAL E UF

df_cadop.columns


Index(['REGISTRO_OPERADORA', 'CNPJ', 'Razao_Social', 'Nome_Fantasia',
       'Modalidade', 'Logradouro', 'Numero', 'Complemento', 'Bairro', 'Cidade',
       'UF', 'CEP', 'DDD', 'Telefone', 'Fax', 'Endereco_eletronico',
       'Representante', 'Cargo_Representante', 'Regiao_de_Comercializacao',
       'Data_Registro_ANS'],
      dtype='object')

In [62]:
df_agrupado = (
    df_cadop
    .groupby(['Razao_Social', 'UF'])
    .size()
    .reset_index(name='quantidade_registros')
)


In [63]:
df_agrupado.head()


,Razao_Social,UF,quantidade_registros
0,18 DE JULHO ADMINISTRADORA DE BENEFÃCIOS LTDA,MG,1
1,2B ODONTOLOGIA OPERADORA DE PLANOS ODONTOLÃGI...,SP,1
2,2CARE OPERADORA DE SAÃDE LTDA.,SP,1
3,A.P.S. ADMINISTRADORA DE BENEFÃCOS LTDA.,SP,1
4,ABERTTA SAÃDE - ASSOCIAÃÃO BENEFICENTE DOS ...,MG,1


In [64]:
df_por_uf = (
    df_cadop
    .groupby('UF')
    .agg(
        quantidade_operadoras=('Razao_Social', 'nunique')
    )
    .reset_index()
    .sort_values('quantidade_operadoras', ascending=False)
)


In [65]:
df_cadop_filtrado = df_cadop[
    df_cadop['Razao_Social'].notna() &
    (df_cadop['Razao_Social'].str.strip() != '')
]

df_agrupado = (
    df_cadop_filtrado
    .groupby(['Razao_Social', 'UF'])
    .size()
    .reset_index(name='quantidade_registros')
)


## Limitações Estruturais dos Dados

Durante o processo de análise e transformação dos dados, foi identificada
uma limitação estrutural que impacta diretamente a possibilidade de integração
entre os conjuntos de dados disponíveis.

### Estrutura dos datasets analisados

- **consolidado_despesas.csv**
  - Colunas: `Trimestre`, `Ano`, `ValorDespesas`
  - Dados financeiros agregados em nível temporal
  - Não possui identificadores de operadoras (CNPJ, Razão Social ou UF)

- **Relatorio_cadop.csv**
  - Contém informações cadastrais das operadoras
  - Inclui campos como CNPJ, Razão Social e UF
  - Dados em nível granular por operadora

### Limitação identificada

Não existe uma chave comum confiável que permita a realização de um JOIN
relacional entre o dataset de despesas consolidadas e o dataset cadastral.
Em especial, o arquivo de despesas não contém CNPJ ou qualquer identificador
institucional que permita associar os valores financeiros a operadoras
específicas.

### Decisão técnica adotada

Optou-se por **não realizar integração relacional entre os datasets**, tratando-os
como fontes analíticas independentes, porém complementares:

- O consolidado de despesas é utilizado para análises macroeconômicas
  e temporais.
- O relatório cadastral (CADOP) é utilizado para análises institucionais
  e geográficas, como agrupamentos por Razão Social e UF.

Essa decisão preserva a integridade dos dados e evita associações incorretas
ou inferências estatísticas inválidas.

### Impactos da limitação

**Prós**
- Mantém consistência e confiabilidade dos dados
- Evita cruzamentos artificiais ou incorretos
- Segue boas práticas de modelagem e governança de dados

**Contras**
- Impede análises financeiras por operadora
- Limita o enriquecimento direto dos dados consolidados

A superação dessa limitação exigiria uma fonte adicional que relacionasse
valores financeiros a identificadores institucionais (ex.: CNPJ).


In [ ]:
#CALCULANDO O TOTAL DE DESPESAS POR OPERADORA E UF



## Limitação no cálculo de despesas por operadora/UF

Apesar de existir um dataset cadastral contendo operadoras e suas respectivas
Unidades Federativas (UF), o dataset de despesas analisado apresenta valores
apenas em nível agregado por período (ano e trimestre).

Não há informação que relacione diretamente valores de despesas a CNPJs,
operadoras ou UFs. Dessa forma, não foi possível calcular o total de despesas
por operadora ou por UF sem introduzir suposições artificiais.

Qualquer tentativa de rateio ou distribuição proporcional dos valores
configuraria uma inferência estatística inválida. Assim, optou-se por não
realizar esse cálculo, preservando a integridade e confiabilidade da análise.


## Limitação no cálculo de médias por operadora e UF

O cálculo da média de despesas por trimestre para cada operadora ou Unidade
Federativa (UF) não pôde ser realizado, pois o dataset de despesas apresenta
valores apenas em nível agregado por período, sem identificação de operadoras.

Não existe relacionamento direto entre os valores financeiros e os dados
cadastrais disponíveis. Qualquer tentativa de distribuição das despesas por
operadora ou UF exigiria suposições artificiais, o que comprometeria a validade
estatística da análise.

Dessa forma, optou-se por calcular médias apenas em nível temporal (ano e
trimestre), preservando a integridade dos resultados.


## Limitação na identificação de operadoras com alta variabilidade de despesas

Foi avaliada a possibilidade de calcular o desvio padrão das despesas por
operadora, com o objetivo de identificar entidades com valores altamente
variáveis ao longo do tempo.

Entretanto, o dataset de despesas disponível apresenta valores apenas em nível
agregado por período (ano e trimestre), sem identificação de operadoras ou UFs.
Dessa forma, não é possível associar a variabilidade das despesas a operadoras
específicas.

Optou-se por calcular o desvio padrão apenas em nível temporal, permitindo a
análise da volatilidade global das despesas, sem comprometer a integridade dos
dados.


In [66]:
#ORDENANDO POR VALOR TOTAL (MAIOR PARA MENOR)

total_por_trimestre = (
    df_consolidado
    .groupby('Trimestre', as_index=False)['ValorDespesas']
    .sum()
    .sort_values(by='ValorDespesas', ascending=False)
)

total_por_trimestre


,Trimestre,ValorDespesas
2,2025Q3,"3,786,024,282,857.07"
1,2025Q2,"2,725,770,948,027.75"
0,2025Q1,"1,633,663,694,933.59"


In [67]:
total_por_ano = (
    df_consolidado
    .groupby('Ano', as_index=False)['ValorDespesas']
    .sum()
    .sort_values(by='ValorDespesas', ascending=False)
)

total_por_ano


,Ano,ValorDespesas
0,2025,"8,145,458,925,818.41"


In [68]:
#SALVANDO RESULTADO EM NOVO CSV

total_por_trimestre.to_csv(
    'despesas_agregadas.csv',
    index=False,
    encoding='utf-8'
)


In [69]:
import os
os.listdir('/content')


['.config',
 'consolidado_despesas.csv',
 'consolidado_despesas.zip',
 'Relatorio_cadop.csv',
 'Relatorio_cadop (1) (1).csv',
 'despesas_agregadas.csv',
 'sample_data']

In [70]:
import zipfile

zip_name = 'Teste_Weydison_Andrade.zip'
csv_name = 'despesas_agregadas.csv'

with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(csv_name)

zip_name


'Teste_Weydison_Andrade.zip'

In [71]:
import os
os.listdir('/content')


['.config',
 'consolidado_despesas.csv',
 'consolidado_despesas.zip',
 'Relatorio_cadop.csv',
 'Teste_Weydison_Andrade.zip',
 'Relatorio_cadop (1) (1).csv',
 'despesas_agregadas.csv',
 'sample_data']